In [3]:
import os
import streamlit as st
import pickle
import time
import langchain
import pprint
import google.generativeai as palm
from langchain.llms import GooglePalm
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import GooglePalmEmbeddings  # Updated
from langchain.vectorstores import FAISS


In [2]:
from dotenv import load_dotenv

load_dotenv()  
google_palm_key = os.getenv('GOOGLE_API_KEY')


In [4]:
llm = GooglePalm(google_api_key=os.getenv('PALM_API_KEY'), temperature=0.9, max_tokens=500)


# (1) Loading The Data 

In [5]:

loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

2

In [6]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)


docs = text_splitter.split_documents(data)
len(docs)

28

In [63]:
docs[9]

Document(page_content='Traders see a 93% chance that the central bank will hold its interest rates at current levels at its September meeting, while chances of a pause in November stand at 57%, according to the CME FedWatch Tool.\n\nFed officials have entered a blackout period, during which they usually do not make public comments, until the policy decision outcome on Sept. 20.\n\nThe S&P 500 climbed 0.67% to end at 4,487.46 points.\n\nThe Nasdaq gained 1.14% at 13,917.89 points, while Dow Jones Industrial Average rose 0.25% to 34,663.72 points.\n\nVolume on U.S. exchanges was relatively light, with 9.3 billion shares traded, compared to an average of 10.0 billion shares over the previous 20 sessions.\n\nOf the 11 S&P 500 sector indexes, nine rose, led by consumer discretionary\xa0(.SPLRCD), up 2.77%, followed by a 1.17% gain in communication services\xa0(.SPLRCL).\n\nQualcomm\xa0(QCOM.O)\xa0advanced 3.9% after the chipmaker signed a new deal with Apple\xa0(AAPL.O)\xa0to supply 5G chip

In [7]:
embeddings = GooglePalmEmbeddings()

vectorindex_palm = FAISS.from_documents(docs, embeddings)


In [77]:
import dill


with open('vectorindex_palm.pkl', 'wb') as f:
    dill.dump(vectorindex_palm, f)

In [80]:
import os
import dill


if os.path.exists("vectorindex_palm.pkl"):
    
    with open("vectorindex_palm.pkl", "rb") as f:
        vectorindex_palm = dill.load(f)
else:
    
    vectorindex_palm = initialize_vectorindex_palm()

In [8]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex_palm.as_retriever())
chain


RetrievalQAWithSourcesChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_documents_chain=MapReduceDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:', template_format='f-string', validate_template=True), llm=GooglePalm(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<module 'google.generativeai' from 'c:\\Users\\bunty\\anaconda3\\envs\\chatbotversion\

In [95]:
# query = "what is the price of Tiago iCNG?"
query = "what are the main features of punch iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what are the main features of punch iCNG?"
}
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Go Ad-Free\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHome\n\nNews\n\nBusiness\n\nTata Motors launches Punch iCNG, price starts at Rs 7.1 lakh\n\nThe Punch iCNG is equipped with the company's proprietary twin-cylinder technology with enhanced safety features like a micro-switch to keep the car switched off at the time of refuelling and thermal incident protection that cuts off CNG supply to the engine and releases gas into the atmosphere, Tata Motors said in a statement.\n\nPTI\n\nAugu

IndexError: list index out of range